In [1]:
import pandas as pd
import os
from data_preparation import load_data
from tqdm import tqdm
tqdm.pandas()

In [2]:
DATA_FOLDER = "/mnt/d/sign-lang-data/"

In [5]:
train = pd.read_csv(os.path.join(DATA_FOLDER, "train_with_nb_frame.csv"))

In [14]:
lst_of_test_parquet = [
    "train_landmark_files/22343/1110225521.parquet",#330
    "train_landmark_files/4718/2748291567.parquet",#5
    "train_landmark_files/49445/2648133604.parquet",#60
    "train_landmark_files/62590/2817812959.parquet" #150
]

In [15]:
#train[train.nb_frame >= 150]

In [45]:
for parquet in lst_of_test_parquet:
    parquet_df = pd.read_parquet(os.path.join(DATA_FOLDER, parquet), engine="pyarrow")
    parquet_df2 = parquet_df[parquet_df.type == 'face']
    # do mean of 10 group in parquet_df2
    nbframe = parquet_df2.groupby("frame").count().shape[0]

    group_size = int(((nbframe - nbframe % 10) / 10))
    rest = nbframe % 10
    print(f"{nbframe} = 10 x {group_size} + {rest}")

    if rest == 0:
        for i in range(10):
            print(parquet_df2.iloc[i*group_size:,:].shape)
    else:
        if group_size == 0:
            print(parquet_df2.shape)
        else:
            for i in range(10):
                print(parquet_df2.iloc[i*group_size:,:].shape)

            print(parquet_df2.iloc[10*group_size+rest:,:].shape)


330 = 10 x 33 + 0
(154440, 7)
(154407, 7)
(154374, 7)
(154341, 7)
(154308, 7)
(154275, 7)
(154242, 7)
(154209, 7)
(154176, 7)
(154143, 7)
6 = 10 x 0 + 6
(2808, 7)
45 = 10 x 4 + 5
(21060, 7)
(21056, 7)
(21052, 7)
(21048, 7)
(21044, 7)
(21040, 7)
(21036, 7)
(21032, 7)
(21028, 7)
(21024, 7)
(21015, 7)
91 = 10 x 9 + 1
(42588, 7)
(42579, 7)
(42570, 7)
(42561, 7)
(42552, 7)
(42543, 7)
(42534, 7)
(42525, 7)
(42516, 7)
(42507, 7)
(42497, 7)
